In [18]:
import pandas as pd
import gensim
import nltk
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer

In [19]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/vc/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
input_file = pd.read_csv('abcnews-date-text.csv')
input_file.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [4]:
data_text = input_file[['headline_text']]
documents = data_text

In [7]:
def lemmatize_token(token):
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(token)

In [16]:
def preprocessing(text):
    token_arr = []
    for tokens in simple_preprocess(text,min_len=3):
        if tokens not in STOPWORDS:
            token_arr.append(lemmatize_token(tokens))
    
    return token_arr
            

In [20]:
sample_doc = documents[:5]
for val in sample_doc['headline_text']:
    token_arr = preprocessing(val)
    print(token_arr)

['aba', 'decides', 'community', 'broadcasting', 'licence']
['act', 'witness', 'aware', 'defamation']
['call', 'infrastructure', 'protection', 'summit']
['air', 'staff', 'aust', 'strike', 'pay', 'rise']
['air', 'strike', 'affect', 'australian', 'traveller']
